# Chapter 19: Training and Deploying TensorFlow Models at Scale

**Tujuan:** Mengetahui cara men‑save dan serve model TensorFlow, membuat TFLite untuk mobile, memanfaatkan GPU / multi‑GPU, dan distribusi training.

---

## 1. Export dan Serving dengan SavedModel & TF‑Serving

- **SavedModel**: format standar untuk deployment.  
- **TensorFlow Serving**: server REST/gRPC untuk model.

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

# Pastikan growth dynamic agar tidak all‑gain VRAM
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

print("GPUs:", gpus)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
GPUs: []


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Buat folder tujuan
import os
model_dir = "my_model"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Contoh dataset: Iris (kecil dan cepat)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

iris = load_iris()
X, y = iris.data.astype("float32"), iris.target
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Bangun model Keras sederhana
model = keras.Sequential([
    layers.Input(shape=(4,)),
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(3, activation="softmax")
])

# Compile
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Latih singkat
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=8
)

# Simpan ke SavedModel di folder "my_model"
tf.saved_model.save(model, "my_model")
print("SavedModel disimpan ke my_model/")

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.3513 - loss: 1.5368 - val_accuracy: 0.3333 - val_loss: 1.5009
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3572 - loss: 1.4040 - val_accuracy: 0.4333 - val_loss: 1.1927
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3916 - loss: 1.1834 - val_accuracy: 0.5333 - val_loss: 1.0621
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6283 - loss: 1.0338 - val_accuracy: 0.6333 - val_loss: 0.9821
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4686 - loss: 0.9609 - val_accuracy: 0.5333 - val_loss: 0.9193
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5355 - loss: 0.8893 - val_accuracy: 0.5667 - val_loss: 0.8686
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6155 - loss: 0.8629 - val_accuracy: 0.6667 - val_loss: 0.8197
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7745 - loss: 0.8167 - val_accuracy: 0.7333 - val_lo

In [3]:
# Install Flask & flask-ngrok
!pip install -q flask flask-ngrok

In [4]:
# Buat Flask server yang load SavedModel via tf.saved_model.load
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import tensorflow as tf
import numpy as np

app = Flask(__name__)
run_with_ngrok(app)  # expose via ngrok

# Load model SavedModel (Opsi B)
loaded = tf.saved_model.load("my_model")
# Biasanya signature default bernama "serving_default"
infer = loaded.signatures["serving_default"]

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json(force=True)
    # Payload: {"instances": [[...],[...],...]}
    instances = np.array(data["instances"], dtype=np.float32)
    # TensorFlow SavedModel expects a tensor input,
    # nama argumen sesuai signature (cek infer.structured_input_signature)
    tf_inputs = tf.constant(instances)
    # Panggil inference
    outputs = infer(tf_inputs)
    # Biasanya output ada di outputs['output_0'] atau nama lain—cek keys()
    # Kita ambil yang pertama:
    out_key = list(outputs.keys())[0]
    preds = outputs[out_key].numpy().tolist()
    return jsonify({"predictions": preds})

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information abo

## 2. Convert ke TFLite untuk Mobile/Embedded
Post‑training quantization kurangi ukuran & latensi.

In [5]:
import tensorflow as tf

# Load SavedModel atau Keras .h5
model = tf.saved_model.load("my_model")

# Buat converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# (Opsional) quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Konversi
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)
print("TFLite model size:", len(tflite_model), "bytes")

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


TFLite model size: 3704 bytes


## 3. Memanfaatkan GPU

In [6]:
import tensorflow as tf

# Pastikan growth dynamic agar tidak all‑gain VRAM
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

print("GPUs:", gpus)


# Model sederhana untuk tes
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(1000,)),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Dummy data
import numpy as np
X = np.random.rand(10000,1000).astype('float32')
y = np.random.randint(0,10,size=(10000,))

model.fit(X, y, epochs=3, batch_size=128)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


GPUs: []


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


Epoch 1/3
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 2.5196
Epoch 2/3
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 2.3163
Epoch 3/3


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


73/79 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.2944

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 2.2943


## 4. Multi‑GPU dengan Distribution Strategies

In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Gunakan MirroredStrategy untuk multi‑GPU
strategy = tf.distribute.MirroredStrategy()
print("Num GPUs:", strategy.num_replicas_in_sync)

with strategy.scope():
    model = keras.Sequential([
        keras.layers.Dense(256, activation='relu', input_shape=(1000,)),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Dummy data
X = np.random.rand(20000,1000).astype('float32')
y = np.random.randint(0,10,size=(20000,))

model.fit(X, y, epochs=5, batch_size=256)

Num GPUs: 1


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
66/79 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0991 - loss: 2.4253

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0989 - loss: 2.4121
Epoch 2/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1038 - loss: 2.3027
Epoch 3/5
11/79 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1080 - loss: 2.3021

...

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


50/79 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1130 - loss: 2.3010

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.1118 - loss: 2.3009


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

## 5. Black‑Box Hyperparameter Tuning dengan KerasTuner

In [8]:
!pip install -q keras-tuner

import keras_tuner as kt
from tensorflow import keras

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(
        units=hp.Int('units', 32, 256, step=32),
        activation='relu', input_shape=(1000,)))
    model.add(keras.layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('lr', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='tuner_dir',
    project_name='chapter19'
)

# Dummy data
import numpy as np
X = np.random.rand(5000,1000).astype('float32')
y = np.random.randint(0,10,size=(5000,))

tuner.search(X, y, epochs=5, validation_split=0.2)
best_model = tuner.get_best_models(num_models=1)[0]
best_hp    = tuner.get_best_hyperparameters(1)[0]
print("Best lr:", best_hp.get('lr'),
      "Best units:", best_hp.get('units'))

Trial 5 Complete [00h 00m 07s]
val_accuracy: 0.09000000357627869

Best val_accuracy So Far: 0.11100000143051147
Total elapsed time: 00h 00m 38s
Best lr: 0.0001 Best units: 256


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# Ringkasan Chapter 19
1. SavedModel + TensorFlow Serving untuk production / REST API.

2. TFLite untuk deployment mobile/embedded (quantization).

3. Manfaatkan GPU dan Distribution Strategies untuk training lebih cepat.

4. KerasTuner untuk black‑box hyperparameter search.